In [1]:
import pandas as pd
import sys
print(sys.version)
import os
import glob
from IPython.display import display, HTML

3.6.0 |Anaconda 4.3.0 (64-bit)| (default, Dec 23 2016, 12:22:00) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


In [17]:
honda=pd.read_csv("/share/pub/OPP-115/annotations/1050_honda.com.csv", header=None)
honda.columns=["ID", "Type", "number", "number2", "section", "category", "annotation", "date", "url"]
honda.head(3)

,ID,Type,number,number2,section,category,annotation,date,url
0,20842,test_category_labeling_highlight_upitt,95,3836,0,Other,"{""Other Type"": {""selectedText"": ""STATEMENT OF ...",12/23/14,http://www.honda.com/site/site_privacy.aspx
1,15695,test_category_labeling_highlight_upitt,103,3836,1,Other,"{""Other Type"": {""selectedText"": ""American Hond...",12/23/14,http://www.honda.com/site/site_privacy.aspx
2,20843,test_category_labeling_highlight_upitt,95,3836,1,Other,"{""Other Type"": {""selectedText"": ""American Hond...",12/23/14,http://www.honda.com/site/site_privacy.aspx


In [18]:
hondaAnnot=honda.loc[:,["section", "category", "annotation"]]

In [9]:
group=list(hondaAnnot.groupby(["category"]))

In [11]:
print (group[1][1])


     section       category                                         annotation
66        14  Data Security  {"Security Measure": {"selectedText": "America...
67        14  Data Security  {"Security Measure": {"selectedText": "The fol...
68        14  Data Security  {"Security Measure": {"selectedText": "In addi...
69        14  Data Security  {"Security Measure": {"selectedText": "However...
70        14  Data Security  {"Security Measure": {"selectedText": "This in...
71        14  Data Security  {"Security Measure": {"selectedText": "In addi...
201       62  Data Security  {"Security Measure": {"selectedText": "America...
203       63  Data Security  {"Security Measure": {"selectedText": "How do ...
204       63  Data Security  {"Security Measure": {"selectedText": "We main...
264       14  Data Security  {"Security Measure": {"selectedText": "America...
265       14  Data Security  {"Security Measure": {"selectedText": "This in...
266       14  Data Security  {"Security Measure": {"

In [15]:
annolist=group[3][1]["annotation"].values
print (group[3][0])
eval(group[3][1]["annotation"].iloc[0])["Purpose"]

First Party Collection/Use


{'endIndexInSegment': -1,
 'selectedText': 'Not selected',
 'startIndexInSegment': -1,
 'value': 'Unspecified'}

In [2]:

## in some texts, not all annotation have the same attributes
abita=pd.read_csv("/share/pub/OPP-115/annotations/571_abita.com.csv", header=None)
abita.columns=["ID", "Type", "number", "number2", "section", "category", "annotation", "date", "url"]

abitaGroup=list(abita[[ "category", "annotation", "section"]].groupby(["category"]))
"""for index in abitaGroup[1][1].index:
  annoRecord=eval(abitaGroup[1][1].ix[index, "annotation"])
  print [len(annoRecord[topic].keys()) for topic in annoRecord]
"""

print (eval(abitaGroup[1][1].iloc[0,1])["Collection Mode"].keys())  # Collection Mode in the first annotation in the category
print (eval(abitaGroup[1][1].iloc[4,1])["Collection Mode"].keys())  # Collection Mode in the 5th annotation in the category

dict_keys(['endIndexInSegment', 'startIndexInSegment', 'value'])
dict_keys(['endIndexInSegment', 'startIndexInSegment', 'selectedText', 'value'])


In [3]:
import json
abitaFP=abitaGroup[1][1]
abitaFPanno=abitaGroup[1][1]["annotation"]
abitaFPannoEg=eval(abitaFPanno.iloc[0])
abitaFPannoEg2=eval(abitaFPanno.iloc[5])
#pd.DataFrame.from_dict([eval(anno) for anno in abitaFPanno], orient="index")
pds1=pd.Series(abitaFPannoEg["Action First-Party"])
pds2=pd.Series(abitaFPannoEg2["Action First-Party"])
pd.DataFrame([pds2, pds1])

,endIndexInSegment,selectedText,startIndexInSegment,value
0,34,We may collect certain information,0,Unspecified
1,531,personal information we collect,500,Unspecified


## Functions for packing annotations

In [4]:
a1=[1,2,3,4]
a2=pd.DataFrame({"cat2":[0,2,4,2], "cat3":[2,3,4,1]})
a3=a2.assign(cat1=a1)
a3

,cat2,cat3,cat1
0,0,2,1
1,2,3,2
2,4,4,3
3,2,1,4


In [42]:
a4=pd.DataFrame({"cat3":[0,2,4,2], "cat2":[2,3,4,1]})
pd.concat([a4, a3], axis=0)

,cat2,cat3
0,2,0
1,3,2
2,4,4
3,1,2


In [53]:
## take two cateogries, put the values of the same key together in a list
def joinDict(origin, newdict):
  for key in origin:
    try:
      origin[key].extend([newdict[key]])
    except KeyError:
      origin[key].append("NaN")
  return origin

## extract all the subcategory (topics) in a category, and gather the values of each topic into a 
## dataframe along with section value
def packCategory(categoryFrame):
  annocollect={}
  annoFrames=[]
  for rownumber in range(categoryFrame.shape[0]):
    anno=eval(categoryFrame["annotation"].iloc[rownumber])
    sectionValue=categoryFrame["section"].iloc[rownumber]
    for topic in anno:
      annocollect.setdefault(topic, [])
      topicAnno=anno[topic]
      topicAnno["section"]=sectionValue
      annocollect[topic].append(pd.Series(topicAnno))

    for topic in anno:
      frame=pd.DataFrame(annocollect[topic])
      frame=frame.loc[:,["selectedText",'section','startIndexInSegment','endIndexInSegment','value']]
      frameTopic=frame.assign(Topic=[topic]*frame.shape[0])
      annoFrames.append(frameTopic)
  categoryFrame=pd.concat(annoFrames, axis=0)
  categoryFrame=categoryFrame.loc[:, ["Topic", "selectedText",'section','startIndexInSegment','endIndexInSegment','value']]
  return categoryFrame

## Concatenate the datafram returned from each category
def packPolicy(policyDataFrame):
  totalresult=[]
  filtered=policyDataFrame.loc[:,[ "category", "annotation", "section"]]
  grouped=list(filtered.groupby(["category"]))
  for group in grouped:
    category=group[0]
    categoryData=group[1]
    categoryFrame=packCategory(categoryData)
    categoryFrame=categoryFrame.assign(Category=[category]*categoryFrame.shape[0])
    totalresult.append(categoryFrame)
  policyAnnoFrame=pd.concat(totalresult, axis=0)
  policyAnnoFrame=policyAnnoFrame.loc[:, ["Category","Topic", "selectedText",'section','startIndexInSegment','endIndexInSegment','value']]
  return policyAnnoFrame

def parseAllAnnotations(fileList):
  fulloutput=[]
  for filepath in fileList:
    website=os.path.splitext(os.path.basename(filepath))[0].split("_")[1]
    annoData=pd.read_csv(filepath, header=None, names=["number", "Type", "ID", "number2", "section", "category", "annotation", "date", "url"])
    parsedAnno=packPolicy(annoData)
    parsedAnno=parsedAnno.assign(Website=[website]*parsedAnno.shape[0])
    fulloutput.append(parsedAnno)
  fullFrame=pd.concat(fulloutput, axis=0)
  fullFrame=fullFrame.loc[:,["Website","Category","Topic", "selectedText",'section','startIndexInSegment','endIndexInSegment','value'] ]
  return fullFrame


In [30]:
eval(abitaGroup[1][1]["annotation"].iloc[0])['Collection Mode']

{'endIndexInSegment': -1, 'startIndexInSegment': -1, 'value': 'not-selected'}

In [12]:
test=packCategory(abitaGroup[1][1])
test.head()

In [27]:
%%time
result=packPolicy(honda)
result.head()

CPU times: user 31.5 s, sys: 52 ms, total: 31.5 s
Wall time: 31.5 s


In [44]:
%%time
## pack all original annotations files
originalAnnoFiles=glob.glob("/share/pub/OPP-115/annotations/*.csv")
packedAnnoOriginal=parseAllAnnotations(originalAnnoFiles)

In [50]:
packedAnnoOriginal=packedAnnoOriginal.

## Organize consolidated annotations

In [55]:
#/Users/NYao/Documents/MIDS/210/Data/OPP-115/consolidation/
hondaConso=pd.read_csv("/share/pub/OPP-115/consolidation/threshold-1.0-overlap-similarity/1050_honda.com.csv")
hondaConso.columns=["Number", "Type", "annotator", "number2", "section", "category", "annotation", "date", "url"]
hondaConso.head(2)

,Number,Type,annotator,number2,section,category,annotation,date,url
0,C549,test_category_labeling_highlight_fordham_aaaaa,121,3836,1,Other,"{""Other Type"": {""selectedText"": ""American Hond...",12/23/14,http://www.honda.com/site/site_privacy.aspx
1,15696,test_category_labeling_highlight_upitt,103,3836,2,Other,"{""Other Type"": {""selectedText"": ""This statemen...",12/23/14,http://www.honda.com/site/site_privacy.aspx


In [219]:
hondaConsoPack=packPolicy(hondaConso)

In [56]:
%%time
consoAnno10=glob.glob("/share/pub/OPP-115/consolidation/threshold-1.0-overlap-similarity/*.csv")
print (len(consoAnno10))
packedAnnoConso10=parseAllAnnotations(consoAnno10)
print(packedAnnoConso10.shape)

115
CPU times: user 17min 52s, sys: 2.61 s, total: 17min 55s
Wall time: 17min 56s


## Consolidated Annotation with 0.5 threshold


In [74]:
%%time
consoAnno05=glob.glob("/share/pub/OPP-115/consolidation/threshold-0.5-overlap-similarity/*.csv")
print(len(consoAnno05))
packedAnnoConso05=parseAllAnnotations(consoAnno05)
print(packedAnnoConso05.shape)

115
(4049555, 8)
CPU times: user 13min 57s, sys: 2.25 s, total: 14min
Wall time: 13min 59s


## Consolidated Annotation with 0.75 threshold


In [75]:
%%time
consoAnno075=glob.glob("/share/pub/OPP-115/consolidation/threshold-0.75-overlap-similarity/*.csv")
print(len(consoAnno075))
packedAnnoConso075=parseAllAnnotations(consoAnno075)
print(packedAnnoConso075.shape)

115
(5275282, 8)
CPU times: user 16min 35s, sys: 1.83 s, total: 16min 37s
Wall time: 16min 36s


## Save results

In [62]:
datastore=pd.HDFStore("/share/pub/Results/DataStore.h5")

In [ ]:
datastore.put("ParsedAnnotation_original", packedAnnoOriginal)

In [ ]:
datastore.put("ParsedAnnotation_Conlidated0.75", packedAnnoConso075)
datastore.put("ParsedAnnotation_Conlidated0.5", packedAnnoConso05)


## Line up selected text with sentences

In [ ]:
def readPolicies(filelist):
  soups={}
  for filename in filelist:
    base=os.path.basename(filename).split("_")[1]
    website=re.sub(".html", '', base)
    soups[website]=BeautifulSoup(open(filename, "r").read(), 'html.parser')
  return soups